In [1]:
import numpy
import pandas
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor
from keras.layers import Dropout
from keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.metrics import mean_squared_error
from functions  import intensity_data
from scipy.io import loadmat
sub=['F1','F2','M1','M2']
# sub = ['M2']
fold=['Fold1','Fold2','Fold3','Fold4']
# fold = ['Fold1']
seg = 'Upper'
a = list(range(98,100))
b = [99.25,99.5,99.75,100]
per = a + b
for p in per:
    for s in sub:
        mat = loadmat('../DNN_Data_dist_DNN_Full/'+seg+'/'+s+'/'+"pxl_np_"+str(p)+".mat")
        np_fold = mat['pxl_np']
        for f in fold: 
            print('perc'+str(p)+'Subject'+s+'and'+f)
            #dataset = pandas.read_csv(seg+'/'+s+'/'+f+'/'+"Grid_Wise_Set_train_"+str(p)+".csv", delimiter=",",header = None)
            #dataset.replace(numpy.inf, numpy.nan)
            #dataset[dataset==numpy.inf] = numpy.nan
            #dataset.fillna(dataset.mean(), inplace=True)
            #dataset = dataset.as_matrix(columns = None)
            ind = fold.index(f)
            np = np_fold[0,ind][1][0][0]
            dataset = loadmat(seg+'/'+s+'/'+f+'/'+"Intensity_Set_"+str(p)+".mat")
            a1= dataset['intensity_profile_frame']
            l = a1.shape[1]
            l1 = int(l/4)
            a = numpy.array(a1[0][0:l-l1])
            a = a.reshape(1,l-l1)
            b = numpy.array(a1[0][l-l1:l])
            b = b.reshape(1,l1)
            a = intensity_data(a)
            b = intensity_data(b)
            print(len(a))
            print(len(b))
            X = a[:,0:np+1]
            Xorg = X.copy()
            X = X[~numpy.isnan(X).any(axis=1)]
            Y = a[:,np+1]
            Y = Y[~numpy.isnan(Xorg).any(axis=1)]
            X1 = b[:,0:np+1]
            X1org = X1.copy()
            X1 = X1[~numpy.isnan(X1).any(axis=1)]
            Y1 = b[:,np+1]
            Y1 = Y1[~numpy.isnan(X1org).any(axis=1)]
            if len(a)<=50000:
                n1 = 64
                n2 = 64
                n3 = 64
                bs = 50
            elif len(a)<=100000:
                n1 = 128
                n2 = 128
                n3 = 128
                bs = 100
            elif len(a)<=200000:
                n1 = 256
                n2 = 128
                n3 = 128
                bs = 150
            elif len(a)<=300000:
                n1 = 256
                n2 = 256
                n3 = 128
                bs = 200
            elif len(a)<=400000:
                n1 = 256
                n2 = 256
                n3 = 256
                bs = 250
            else:
                n1 = 512
                n2 = 256
                n3 = 256
                bs = 300
#             if len(a)<=40000:
#                 n1 = 64
#                 n2 = 64
#                 n3 = 64
#             else:
#                 n1 = 128
#                 n2 = 128
#                 n3 = 128
            def base_model():
                model = Sequential()
                model.add(Dense(n1,activation="relu", kernel_initializer="normal", input_dim=np+1))
                model.add(Dropout(0.15))
#                 if l > 20000:
                model.add(Dense(n2, activation="relu", kernel_initializer="normal"))
                model.add(Dropout(0.15))
#                 if l > 50000:
                model.add(Dense(n3, activation="relu", kernel_initializer="normal"))
                model.add(Dropout(0.15))  
                model.add(Dense(1, activation="relu", kernel_initializer="normal"))
                model.compile(loss='mean_squared_error',optimizer='adam')
                return model
            fname=(seg+'/'+s+'/'+f+'/'+'Model_GridWise_dev_' + str(p) + '.hdf5')
            callbacks = [EarlyStopping(monitor='val_loss', patience=4, min_delta = 0.00001), ModelCheckpoint(fname, monitor='val_loss', verbose=0, save_best_only=True, mode='auto', period=1)]
            estimator = KerasRegressor( build_fn = base_model)
            estimator.fit(X, Y, validation_data=(X1,Y1), epochs=100, batch_size=bs, callbacks=callbacks)
            a = []
            b = []
            X1 = []
            Y1 = []
            X1org = []
            Xorg = []
            X = []
            Y = []
            a1 = []
            dataset = []            
print("TRAINING COMPLETE!!")

Using TensorFlow backend.


perc98SubjectF1andFold1
191434
61461
Train on 171044 samples, validate on 55415 samples
Epoch 1/100
171044/171044 [==============================] - 7s 40us/step - loss: 0.0559 - val_loss: 0.0501
Epoch 2/100
171044/171044 [==============================] - 6s 37us/step - loss: 0.0469 - val_loss: 0.0485
Epoch 3/100
171044/171044 [==============================] - 6s 38us/step - loss: 0.0445 - val_loss: 0.0472
Epoch 4/100
171044/171044 [==============================] - 6s 37us/step - loss: 0.0429 - val_loss: 0.0445
Epoch 5/100
171044/171044 [==============================] - 6s 38us/step - loss: 0.0416 - val_loss: 0.0436
Epoch 6/100
171044/171044 [==============================] - 6s 36us/step - loss: 0.0405 - val_loss: 0.0446
Epoch 7/100
171044/171044 [==============================] - 6s 37us/step - loss: 0.0395 - val_loss: 0.0419
Epoch 8/100
171044/171044 [==============================] - 6s 35us/step - loss: 0.0388 - val_loss: 0.0422
Epoch 9/100
171044/171044 [=====================

88237/88237 [==============================] - 5s 62us/step - loss: 0.0240 - val_loss: 0.0291
Epoch 29/100
88237/88237 [==============================] - 6s 62us/step - loss: 0.0239 - val_loss: 0.0281
Epoch 30/100
88237/88237 [==============================] - 6s 70us/step - loss: 0.0238 - val_loss: 0.0296
Epoch 31/100
88237/88237 [==============================] - 7s 75us/step - loss: 0.0236 - val_loss: 0.0286
Epoch 32/100
88237/88237 [==============================] - 6s 70us/step - loss: 0.0234 - val_loss: 0.0279
Epoch 33/100
88237/88237 [==============================] - 6s 68us/step - loss: 0.0231 - val_loss: 0.0279
Epoch 34/100
88237/88237 [==============================] - 6s 70us/step - loss: 0.0228 - val_loss: 0.0299
Epoch 35/100
88237/88237 [==============================] - 7s 76us/step - loss: 0.0229 - val_loss: 0.0278
Epoch 36/100
88237/88237 [==============================] - 6s 73us/step - loss: 0.0227 - val_loss: 0.0278
Epoch 37/100
88237/88237 [========================

78930/78930 [==============================] - 4s 51us/step - loss: 0.0293 - val_loss: 0.0324
Epoch 28/100
78930/78930 [==============================] - 4s 50us/step - loss: 0.0292 - val_loss: 0.0329
Epoch 29/100
78930/78930 [==============================] - 4s 51us/step - loss: 0.0290 - val_loss: 0.0321
Epoch 30/100
78930/78930 [==============================] - 4s 49us/step - loss: 0.0290 - val_loss: 0.0336
Epoch 31/100
78930/78930 [==============================] - 4s 50us/step - loss: 0.0286 - val_loss: 0.0322
Epoch 32/100
78930/78930 [==============================] - 4s 48us/step - loss: 0.0288 - val_loss: 0.0334
Epoch 33/100
78930/78930 [==============================] - 4s 49us/step - loss: 0.0285 - val_loss: 0.0326
perc98SubjectF2andFold1
196543
63204
Train on 120867 samples, validate on 39082 samples
Epoch 1/100
120867/120867 [==============================] - 8s 65us/step - loss: 0.0548 - val_loss: 0.0433
Epoch 2/100
120867/120867 [==============================] - 7s 59us

Epoch 21/100
140117/140117 [==============================] - 12s 85us/step - loss: 0.0315 - val_loss: 0.0311
Epoch 22/100
140117/140117 [==============================] - 12s 84us/step - loss: 0.0314 - val_loss: 0.0298
Epoch 23/100
140117/140117 [==============================] - 12s 83us/step - loss: 0.0311 - val_loss: 0.0304
Epoch 24/100
140117/140117 [==============================] - 12s 82us/step - loss: 0.0309 - val_loss: 0.0297
Epoch 25/100
140117/140117 [==============================] - 12s 83us/step - loss: 0.0307 - val_loss: 0.0300
Epoch 26/100
140117/140117 [==============================] - 12s 83us/step - loss: 0.0305 - val_loss: 0.0301
Epoch 27/100
140117/140117 [==============================] - 12s 83us/step - loss: 0.0304 - val_loss: 0.0299
Epoch 28/100
140117/140117 [==============================] - 12s 83us/step - loss: 0.0302 - val_loss: 0.0302
perc98SubjectF2andFold3
188684
61062
Train on 116871 samples, validate on 37532 samples
Epoch 1/100
116871/116871 [=====

...

139865/139865 [==============================] - 18s 131us/step - loss: 0.0322 - val_loss: 0.0294
Epoch 22/100
139865/139865 [==============================] - 20s 143us/step - loss: 0.0320 - val_loss: 0.0283
Epoch 23/100
139865/139865 [==============================] - 20s 140us/step - loss: 0.0319 - val_loss: 0.0288
Epoch 24/100
139865/139865 [==============================] - 20s 142us/step - loss: 0.0315 - val_loss: 0.0295
Epoch 25/100
139865/139865 [==============================] - 20s 140us/step - loss: 0.0314 - val_loss: 0.0296
Epoch 26/100
139865/139865 [==============================] - 20s 144us/step - loss: 0.0311 - val_loss: 0.0291
perc100SubjectF2andFold4
210055
71816
Train on 124346 samples, validate on 41774 samples
Epoch 1/100
124346/124346 [==============================] - 24s 195us/step - loss: 0.0486 - val_loss: 0.0383
Epoch 2/100
124346/124346 [==============================] - 18s 145us/step - loss: 0.0369 - val_loss: 0.0366
Epoch 3/100
124346/124346 [===========

137732/137732 [==============================] - 17s 126us/step - loss: 0.0323 - val_loss: 0.0309
Epoch 5/100
137732/137732 [==============================] - 19s 137us/step - loss: 0.0309 - val_loss: 0.0327
Epoch 6/100
137732/137732 [==============================] - 16s 118us/step - loss: 0.0298 - val_loss: 0.0290
Epoch 7/100
137732/137732 [==============================] - 18s 127us/step - loss: 0.0286 - val_loss: 0.0276
Epoch 8/100
137732/137732 [==============================] - 18s 132us/step - loss: 0.0276 - val_loss: 0.0266
Epoch 9/100
137732/137732 [==============================] - 18s 128us/step - loss: 0.0271 - val_loss: 0.0256
Epoch 10/100
137732/137732 [==============================] - 17s 125us/step - loss: 0.0265 - val_loss: 0.0248
Epoch 11/100
137732/137732 [==============================] - 17s 127us/step - loss: 0.0258 - val_loss: 0.0247
Epoch 12/100
137732/137732 [==============================] - 16s 116us/step - loss: 0.0253 - val_loss: 0.0250
Epoch 13/100
137732

158852/158852 [==============================] - 21s 132us/step - loss: 0.0239 - val_loss: 0.0233
Epoch 14/100
158852/158852 [==============================] - 21s 131us/step - loss: 0.0236 - val_loss: 0.0222
Epoch 15/100
158852/158852 [==============================] - 21s 130us/step - loss: 0.0234 - val_loss: 0.0221
Epoch 16/100
158852/158852 [==============================] - 20s 128us/step - loss: 0.0232 - val_loss: 0.0231
Epoch 17/100
158852/158852 [==============================] - 21s 131us/step - loss: 0.0229 - val_loss: 0.0213
Epoch 18/100
158852/158852 [==============================] - 21s 132us/step - loss: 0.0227 - val_loss: 0.0208
Epoch 19/100
158852/158852 [==============================] - 21s 131us/step - loss: 0.0225 - val_loss: 0.0208
Epoch 20/100
158852/158852 [==============================] - 21s 132us/step - loss: 0.0223 - val_loss: 0.0213
Epoch 21/100
158852/158852 [==============================] - 21s 131us/step - loss: 0.0221 - val_loss: 0.0210
Epoch 22/100
1

168980/168980 [==============================] - 20s 119us/step - loss: 0.0214 - val_loss: 0.0227
Epoch 25/100
168980/168980 [==============================] - 20s 118us/step - loss: 0.0212 - val_loss: 0.0224
Epoch 26/100
168980/168980 [==============================] - 19s 113us/step - loss: 0.0210 - val_loss: 0.0222
Epoch 27/100
168980/168980 [==============================] - 20s 116us/step - loss: 0.0209 - val_loss: 0.0224
Epoch 28/100
168980/168980 [==============================] - 20s 117us/step - loss: 0.0207 - val_loss: 0.0214
Epoch 29/100
168980/168980 [==============================] - 19s 113us/step - loss: 0.0207 - val_loss: 0.0229
Epoch 30/100
168980/168980 [==============================] - 20s 116us/step - loss: 0.0205 - val_loss: 0.0223
Epoch 31/100
168980/168980 [==============================] - 20s 121us/step - loss: 0.0204 - val_loss: 0.0215
Epoch 32/100
168980/168980 [==============================] - 19s 112us/step - loss: 0.0203 - val_loss: 0.0213
Epoch 33/100
1

161361/161361 [==============================] - 20s 121us/step - loss: 0.0246 - val_loss: 0.0227
Epoch 26/100
161361/161361 [==============================] - 19s 120us/step - loss: 0.0244 - val_loss: 0.0228
Epoch 27/100
161361/161361 [==============================] - 20s 122us/step - loss: 0.0241 - val_loss: 0.0224
Epoch 28/100
161361/161361 [==============================] - 20s 121us/step - loss: 0.0241 - val_loss: 0.0225
Epoch 29/100
161361/161361 [==============================] - 20s 127us/step - loss: 0.0240 - val_loss: 0.0231
Epoch 30/100
161361/161361 [==============================] - 20s 124us/step - loss: 0.0238 - val_loss: 0.0228
Epoch 31/100
161361/161361 [==============================] - 19s 120us/step - loss: 0.0238 - val_loss: 0.0220
Epoch 32/100
161361/161361 [==============================] - 21s 130us/step - loss: 0.0235 - val_loss: 0.0231
Epoch 33/100
161361/161361 [==============================] - 20s 123us/step - loss: 0.0234 - val_loss: 0.0221
Epoch 34/100
1

In [24]:
print(X.shape)
X = X[~numpy.isnan(X).any(axis=1)]

(12957, 3)


In [25]:
X.shape

(12048, 3)

In [5]:
numpy.isnan(X).any(axis=1)

array([False, False, False, ..., False, False, False])